In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import os

In [2]:
file = '/home/jarlene/Desktop/stock/AMEX.txt'

In [3]:
data = pd.read_csv(file, sep='\t')

In [4]:
data

,Symbol,Description
0,AAA,First Priority Clo Bond ETF
1,AAAU,GS Physical Gold ETF
2,AAMC,Altisource Asset Management Corp Com
3,AAU,Almaden Minerals
4,ABEQ,Absolute Core Strategy ETF
...,...,...
2961,ZOM,Zomedica Pharmaceuticals Corp
2962,ZROZ,25+ Year Zero Cpn U.S. Trsy Idx ETF Pimco
2963,ZSB,Uscf Sustainable Battery Metals Strategy Fund
2964,ZSL,Ultrashort Silver -2X ETF


In [4]:
def preprocess(path, threadhold=90, max_len=60):
    data = pd.read_csv(path, sep='\t')
    trade_date_lenth = 6
    cols_len = len(data.columns) - 1
    trade_num = int(cols_len/trade_date_lenth)
    dataes = [data.iloc[:, 1+i*trade_date_lenth:trade_date_lenth *
                        (i+1) + 1] for i in range(trade_num)]
    for d in dataes:
        d['date'] = data['date']
        d.columns = ['open', 'high', 'low',
                     'close', 'adj.close', 'volume', 'date']

    val_datas = []
    for d in dataes:
        d = d.dropna(subset=['open'])
        if len(d) >= threadhold:
            val_datas.append(d)

    datas = []
    for d in val_datas:
        d = d.drop(['date'], axis=1)
        idx = np.random.randint(0, len(d) - max_len - 1, (1,)).item()
        datas.extend([d.loc[i:i+max_len+1,] for i in range(idx)])

    res = [i.to_numpy() for i in datas]
    res = np.array(res)
    np.save(path.replace('.csv', ''), res)

In [3]:
file = '../data/stock/NASDAQ.csv'

In [5]:
preprocess(file)